In [8]:
# Google Colab + Github Connect
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git config --global user.email "silasskim@gmail.com"
!git config --global user.name "Rafflesia"

In [4]:
from getpass import getpass
import os
username = "your_github_username"
token = getpass('Enter your GitHub token: ')
repo_name = "your_repository_name"

!git clone https://{token}@github.com/{username}/{repo_name}.git

fatal: destination path 'Py_Selenm_PPA' already exists and is not an empty directory.


In [5]:
%cd /content/{repo_name}

/content/Py_Selenm_PPA


In [11]:
# The main code Starts
!pip install selenium
!pip install webdriver_manager
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 25.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,653 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu j

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [13]:
# Define Process
def extract_market_data(target_date='2025-01-29', target_contract='FTS PPA 26/35'):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)

    try:
        url = f"https://www.omip.pt/en/dados-mercado?date={target_date}&product=EL&zone=ES&instrument=FTS&maturity=PPA"
        driver.get(url)
        print(f"Page loaded successfully for date: {target_date}")
        time.sleep(5)

        rows = driver.find_elements(By.CSS_SELECTOR, "tr.odd, tr.even")
        all_data = []

        print(f"\nLooking for contract: {target_contract}")

        for row in rows:
            try:
                contract_element = row.find_element(By.TAG_NAME, "strong")
                if contract_element:
                    contract_name = contract_element.text
                    if contract_name == target_contract:
                        print(f"Processing contract: {contract_name}")

                        chart_button = row.find_element(By.CSS_SELECTOR, "a.chart-down")
                        chart_url = chart_button.get_attribute("href")

                        full_history_url = chart_url.replace('/0/0', '/0/1')
                        print(f"Full history URL: {full_history_url}")

                        # Full History
                        driver.execute_script(f"window.open('{full_history_url}', '_blank');")
                        time.sleep(3)
                        driver.switch_to.window(driver.window_handles[-1])

                        chart_div = driver.find_element(By.CLASS_NAME, "charts-highchart")
                        chart_data = chart_div.get_attribute("data-chart")

                        import json
                        data = json.loads(chart_data)

                        # Extract
                        price_series = next(series for series in data['series'] if series['name'] == 'Price (€)')

                        # DataFrame
                        for point in price_series['data']:
                            timestamp = point[0] / 1000
                            price = point[1]
                            date = pd.to_datetime(timestamp, unit='s')

                            all_data.append({
                                'Date': date,
                                'Price': price
                            })

                        # Close tab and switch back
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        break

            except Exception as e:
                print(f"Error processing row: {e}")
                continue

        # Create DataFrame
        df = pd.DataFrame(all_data)
        df = df.sort_values('Date')

        print(f"\nDate range for {target_contract}:")
        print("Earliest date:", df['Date'].min())
        print("Latest date:", df['Date'].max())

        return df

    finally:
        driver.quit()

# Extract wanted Date / Type
target_date = '2025-01-29'
target_contract = 'FTS PPA 26/35'
df = extract_market_data(target_date, target_contract)

print("\nExtracted Data:")
print(df)

# Export CSV
filename = f'ppahistory{target_contract}{target_date}.csv'.replace('/', '')
df.to_csv(filename, index=False)
from google.colab import files
files.download(filename)

Page loaded successfully for date: 2025-01-29

Looking for contract: FTS PPA 26/35
Error processing row: Message: no such element: Unable to locate element: {"method":"tag name","selector":"strong"}
  (Session info: chrome=133.0.6943.53); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5af3a852c09a <unknown>
#1 0x5af3a7ff38b0 <unknown>
#2 0x5af3a8044b99 <unknown>
#3 0x5af3a8044d21 <unknown>
#4 0x5af3a8038616 <unknown>
#5 0x5af3a806a9ad <unknown>
#6 0x5af3a803850a <unknown>
#7 0x5af3a806ab4e <unknown>
#8 0x5af3a8090b8b <unknown>
#9 0x5af3a806a753 <unknown>
#10 0x5af3a803638e <unknown>
#11 0x5af3a8037b51 <unknown>
#12 0x5af3a84f576b <unknown>
#13 0x5af3a84f96f2 <unknown>
#14 0x5af3a84e18fc <unknown>
#15 0x5af3a84fa2e4 <unknown>
#16 0x5af3a84c59cf <unknown>
#17 0x5af3a851acd8 <unknown>
#18 0x5af3a851aeb6 <unknown>
#19 0x5af3a852af16 <unknown>
#20 0x79d48b519ac3 <unknown>

Erro

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
! git add .

In [18]:
!git commit -m "Main Code Commit"

[main fe8d723] Main Code Commit
 1 file changed, 27 insertions(+)
 create mode 100644 ppahistoryFTS PPA 26352025-01-29.csv


In [19]:
!git push origin main

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 512 bytes | 512.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/joyful128/Py_Selenm_PPA.git
   03398b2..fe8d723  main -> main


In [20]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [21]:
!pwd
!ls

/content/Py_Selenm_PPA
'ppahistoryFTS PPA 26352025-01-29.csv'	 PPA_Selenium_Code.ipynb   README.md


In [22]:
!mv "/content/Selenium_PPA_Code.ipynb" "/content/Py_Selenm_PPA/PPA_Selenium_Code.ipynb"

mv: cannot stat '/content/Selenium_PPA_Code.ipynb': No such file or directory


In [23]:
!git add Selenium_PPA_Code.ipynb
!git commit -m "Add Selenium notebook"
!git push

fatal: pathspec 'Selenium_PPA_Code.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [24]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [25]:
!git rm PPA_Selenium_Code.ipynb
!git commit -m "Remove old notebook"
!git push

rm 'PPA_Selenium_Code.ipynb'
[main b6b87ff] Remove old notebook
 1 file changed, 53 deletions(-)
 delete mode 100644 PPA_Selenium_Code.ipynb
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 295 bytes | 295.00 KiB/s, done.
Total 2 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/joyful128/Py_Selenm_PPA.git
   fe8d723..b6b87ff  main -> main


In [26]:
!pwd
!ls
!git status

/content/Py_Selenm_PPA
'ppahistoryFTS PPA 26352025-01-29.csv'	 README.md
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [27]:
!find /content -name "Selenium_PPA_Code.ipynb"

/content/drive/MyDrive/Colab Notebooks/Selenium_PPA_Code.ipynb


In [28]:
!cp /content/Selenium_PPA_Code.ipynb /content/Py_Selenm_PPA/

cp: cannot stat '/content/Selenium_PPA_Code.ipynb': No such file or directory
